In [17]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [18]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_6월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(548094, 2) Index(['split_str', 'org_idx'], dtype='object')
548094


In [19]:
df.isnull().sum()

split_str    3
org_idx      0
dtype: int64

In [20]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [21]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        if a and m:
            morphs.append(m) # 형태소 추가
            pos_list.append(a) # 형태소/품사 추가
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
                # 품사가 조사면 조사 리스트에 단어 추가.
                elif y == 'Josa':
                    josas.append(x)
        else:
            morphs.append(None) # 형태소 추가
            pos_list.append(None) # 형태소/품사 추가
    except:
        pass

 77%|███████▋  | 422755/548094 [7:35:41<4:01:57,  8.63it/s]  

In [6]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['개봉', '예정', '영화', '신작', '영화', '추천', '신작', '영화', '추천', '극장']
['잡다', '적다', '섞다', '주다', '어리다', '불다', '느끼다', '적다', '크다', '안기다']
['어느덧다', '거대하다', '신비하다', '있다', '진하다', '반갑다', '기대하다', '촉촉하다', '어지간하다', '재밌다']
['가', '에', '는', '의', '가', '의', '와', '을', '죠', '의']


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

119334


,명사,명사빈도
0,것,113633
1,수,73383
2,그,60142
3,이,54410
4,곳,42515
...,...,...
119329,금섬,1
119330,몬테베르데,1
119331,올도완,1
119332,세석기,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1777


,동사,동사빈도
0,하다,512247
1,되다,105342
2,보다,62954
3,않다,47485
4,되어다,44222
...,...,...
1772,잘근거리다,1
1773,씨부리다,1
1774,짤깍거리다,1
1775,나불대다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1782


,형용사,형용사빈도
0,있다,238681
1,없다,44349
2,좋다,43390
3,같다,38599
4,이다,31650
...,...,...
1777,차근차근하다,1
1778,살다,1
1779,섬려하다,1
1780,들큼하다,1


In [10]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/6월/나무문화_6월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

379


,조사,조사빈도
0,을,410997
1,의,329384
2,이,317004
3,에,302504
4,를,202736
...,...,...
374,까지만은,1
375,에게서가,1
376,께옵서,1
377,로서나,1


In [11]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_6월_단어종합.txt", index=False, encoding='UTF-8')

In [12]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

ValueError: Length of values (548091) does not match length of index (548094)

In [ ]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_6월_형태소문석.txt", index=False, encoding='UTF-8')